In [6]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
import os
#from keras.utils import plot_model
from keras.utils.vis_utils import plot_model
import cv2

#import tensorflow as tf
#import tensorflow.compat.v1 as tf
np.random.seed(1337)  # for reproducibility

In [7]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils
#from keras.optimizers import SGD, Adadelta, Adagrad
from tensorflow.keras.optimizers import SGD ,Adadelta, Adagrad

from six.moves import cPickle as pickle

In [8]:
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

Device mapping: no known devices.


In [9]:
import warnings
warnings.simplefilter("ignore")

In [10]:
#os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
os.environ["PATH"] += os.pathsep + ''

pickle_files = [r'C:\Users\chida\OneDrive\Desktop\back\Driver Drowsiness Monitoring using\Driver Drowsiness Monitoring using\M2/open_eyes.pickle', r'C:\Users\chida\OneDrive\Desktop\back\Driver Drowsiness Monitoring using\Driver Drowsiness Monitoring using\M2/closed_eyes.pickle']
i = 0
for pickle_file in pickle_files:
    with open(pickle_file, 'rb') as f:
        save = pickle.load(f)
        if i == 0:
            train_dataset = save['train_dataset']
            train_labels = save['train_labels']
            test_dataset = save['test_dataset']
            test_labels = save['test_labels']
        else:
            print("here")
            train_dataset = np.concatenate((train_dataset, save['train_dataset']))
            train_labels = np.concatenate((train_labels, save['train_labels']))
            test_dataset = np.concatenate((test_dataset, save['test_dataset']))
            test_labels = np.concatenate((test_labels, save['test_labels']))
        del save  # hint to help gc free up memory
    i += 1

here


In [11]:
print('Training set', train_dataset.shape, train_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (3876, 24, 24, 1) (3876, 1)
Test set (970, 24, 24, 1) (970, 1)


In [12]:
batch_size = 30
nb_classes = 1
epochs = 10

In [13]:
X_train = train_dataset
X_train = X_train.reshape(X_train.shape[0],24,24,1)
Y_train = train_labels
print(X_train[0])
X_test = test_dataset
X_test = X_test.reshape(X_test.shape[0],24,24,1)
Y_test = test_labels

[[[0.5489647 ]
  [0.51367414]
  [0.47838354]
  [0.47838354]
  [0.5058318 ]
  [0.5175953 ]
  [0.50191057]
  [0.47446236]
  [0.49798942]
  [0.49798942]
  [0.49798942]
  [0.5097529 ]
  [0.52543765]
  [0.5489647 ]
  [0.5685706 ]
  [0.5842553 ]
  [0.5842553 ]
  [0.6117035 ]
  [0.6469941 ]
  [0.6705212 ]
  [0.67444235]
  [0.6783635 ]
  [0.6822847 ]
  [0.68620586]]

 [[0.5175953 ]
  [0.49798942]
  [0.47446236]
  [0.46662   ]
  [0.47054118]
  [0.47446236]
  [0.46662   ]
  [0.45877764]
  [0.45877764]
  [0.46269882]
  [0.46662   ]
  [0.47446236]
  [0.47838354]
  [0.48622587]
  [0.49014705]
  [0.49406824]
  [0.5411224 ]
  [0.5646494 ]
  [0.59994   ]
  [0.6313094 ]
  [0.6587576 ]
  [0.68620586]
  [0.7136541 ]
  [0.73326   ]]

 [[0.49014705]
  [0.4823047 ]
  [0.46662   ]
  [0.44701412]
  [0.4313294 ]
  [0.42740825]
  [0.4313294 ]
  [0.43525058]
  [0.42348707]
  [0.4313294 ]
  [0.44309294]
  [0.4509353 ]
  [0.45877764]
  [0.45877764]
  [0.4548565 ]
  [0.4509353 ]
  [0.51367414]
  [0.5293588 ]
  [0.5

In [14]:
# print shape of data while model is building
print("{1} train samples, {4} channel{0}, {2}x{3}".format("" if X_train.shape[1] == 1 else "s", *X_train.shape))
print("{1}  test samples, {4} channel{0}, {2}x{3}".format("" if X_test.shape[1] == 1 else "s", *X_test.shape))

3876 train samples, 1 channels, 24x24
970  test samples, 1 channels, 24x24


In [15]:
# input image dimensions
_, img_channels, img_rows, img_cols = X_train.shape

In [16]:
model = Sequential()

model.add(Convolution2D(32, (3, 3), padding='same',
                        input_shape=(24,24,1),data_format='channels_last'))
model.add(Activation('relu'))
model.add(Convolution2D(24, (3, 3), data_format='channels_last'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, (3, 3), padding='same', data_format='channels_last'))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('sigmoid'))

In [17]:
# let's train the model using SGD + momentum (how original).
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [18]:
model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, verbose=2, validation_data=(X_test, Y_test))

Train on 3876 samples, validate on 970 samples
Epoch 1/10
3876/3876 - 4s - loss: 0.6584 - acc: 0.6022 - val_loss: 0.6015 - val_acc: 0.6639 - 4s/epoch - 1ms/sample
Epoch 2/10
3876/3876 - 4s - loss: 0.5992 - acc: 0.6860 - val_loss: 0.5567 - val_acc: 0.7216 - 4s/epoch - 983us/sample
Epoch 3/10
3876/3876 - 4s - loss: 0.5092 - acc: 0.7423 - val_loss: 0.4569 - val_acc: 0.7773 - 4s/epoch - 1ms/sample
Epoch 4/10
3876/3876 - 4s - loss: 0.3884 - acc: 0.8264 - val_loss: 0.3288 - val_acc: 0.8773 - 4s/epoch - 1ms/sample
Epoch 5/10
3876/3876 - 4s - loss: 0.2880 - acc: 0.8824 - val_loss: 0.2263 - val_acc: 0.9247 - 4s/epoch - 1ms/sample
Epoch 6/10
3876/3876 - 6s - loss: 0.2296 - acc: 0.9082 - val_loss: 0.1955 - val_acc: 0.9278 - 6s/epoch - 1ms/sample
Epoch 7/10
3876/3876 - 6s - loss: 0.1909 - acc: 0.9249 - val_loss: 0.1526 - val_acc: 0.9454 - 6s/epoch - 1ms/sample
Epoch 8/10
3876/3876 - 5s - loss: 0.1799 - acc: 0.9350 - val_loss: 0.1601 - val_acc: 0.9361 - 5s/epoch - 1ms/sample
Epoch 9/10
3876/3876 - 

In [19]:
model.summary

<bound method Model.summary of <keras.engine.sequential.Sequential object at 0x000001720F453F70>>

In [20]:
score = model.evaluate(X_test, Y_test,  verbose=1)

In [21]:
score

[0.13405924555797552, 0.94639176]

In [22]:
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.13405924555797552
Test accuracy: 0.94639176
